<center>
<img src="https://tensorflowkorea.files.wordpress.com/2020/12/4.-e18492e185a9e186abe1848ce185a1-e18480e185a9e186bce18487e185aee18492e185a1e18482e185b3e186ab-e18486e185a5e18489e185b5e186abe18485e185a5e18482e185b5e186bce18483e185b5e186b8e18485e185a5e.png?w=972" width="200" height="200"><br>
</center>

## 07-2 심층 신경망

### - 2개의 층

다시 케라스 API를 사용하여 패션 MNIST 데이터셋을 불러오자.

In [30]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

그다음 이미지의 픽셀값을 0 ~ 255 범위에서 0 ~ 1 사이로 변환하고, 28 x 28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼친다. 마지막 사이킷런의 train_test_split() 함수로 훈련 세트와 검증 세트로 나눈다. 여기까지는 1절에서 했던 것과 같습니다. 다음 코드가 이해되지 않는다면 1절을 다시 학습해 보자.

In [31]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

이제 인공 신경망 모델에 층을 2개 추가해 보자. 1절에서 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된다. 이렇게 입력층과 출력층 사이에 있는 모든 층을 은닉층이라고 부른다. 은닉층에는 활성화 함수가 있다. 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다. 앞서 출력층에 적용했던 소프트맥스 함수도 활성화 함수이다. 출력층에 적용하는 활성화 함수는 종류가 제한되어 있다. 이진 분류일 경우 시그모이드 함수를 사용하고 다중 분류일 경우 소프트맥스 함수를 사용한다. 이에 비해 은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루 함수등을 사용한다.

그런데 은닉층에 왜 활성화 함수를 적용할까? 은닉층에선 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈이다. 선형 계산을 적당하게 비선형적으로 비틀어 주어야 한다. 그래야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있다.

많이 사용하는 활성화 함수 중 하나는 4장에서 배웠던 시그모이드 함수이다. 이 함수는 뉴런의 출력 z값으 0과 1사이로 압축한다. 이제 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어 보자. 앞서 언급했듯이 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해 주어야 한다.

In [32]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dens1이 은닉층이고 100개의 뉴런을 가진 밀집층이다. 활성화 함수를 sigmoid로 지정했고 input_shape 매개변수에서 입력의 크기를 (784,)로 지정했다. 은닉층의 뉴런 개수를 정하는데는 특별한 기준이 없다. 여기서 한 가지 제약 사항이 있다면 적어도 출력층의 뉴런보다는 많게 만들어야 한다. 클래스 10개에 대한 확률을 예측해야 하는데 이전 은닉층의 뉴런이 10개보다 적다면 부족한 정보가 전달될 것이다.

그다음 dense2는 출력층이다. 10개의 클래스를 분류하므로 10개의 뉴런을 두었고 활성화 함수는 소프트맥스 함수로 지정했다.

### - 심층 신경망 만들기

이제 앞에서 만든 dense1과 dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어 보자.

In [33]:
model = keras.Sequential([dense1, dense2])

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 위와 같이 리스트로 만들어 전달한다. 여기서 주의할 것은 출력츠을 가장 마지막에 두어야 한다는 것이다. 이 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 한다.

인공 신경망의 강력한 성능은 바로 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나온다. 물론 2개 이상의 층을 추가할 수도 있다.

케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.

In [34]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫줄에는 모델의 이름이 나온다. 그다음 이 모델에 들어 있는 층이 순서대로 나열된다. 이 순서는 맨 처음 추가한 은닉층에서 출력층의 순서대로 나열된다. 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다. 층을 만들 때 name 매개변수로 이름을 지정할 수 있다. 층 이름을 지정하지 않으면 케라스가 자동으로 'dense'라고 이름을 붙인다.

출력 크기를 보면 (None, 100)이다. 첫 번째 차원은 샘플의 개수를 나타낸다. 샘플의 개수가 아직 정의되어 있지 않기 때문에 None이다. 왜 그럴까? 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행한다. 바로 미니배치 경사 하강법이다. 

케라스의 기본 미니배치 크기는 32개이다. 이 값은 fit() 메서드에서 batch_size 매개변수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부른다.

두 번째 100은 은닉층의 뉴런 개수를 100개로 두어서 100개의 출력이 나오는 것이다.즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.

마지막으로 모델 파라미터 개수가 출력된다. 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다. 그리고 뉴런마다 1개의 절편이 있다. 따라서 총 78500개 이다.

두 번째 층의 출력 크기는 (None,10)이다. 배치 차원은 동일하게 None이고 출력 뉴런 개수가 10개이기 때문이다. 이 층의 모델 파라미터 개수는 1010개이다.

summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79510으로 나온다. 그 아래 훈련되지 않는 파라미터는 0으로 나온다. 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있다. 이런 층의 파라미터 개수가 여기에 나온다.

### - 층을 추가하는 다른 방법

모델을 훈련하기 전에 Seqeuntial 클래스에 층을 추가하는 다른 방법을 알아보자. 앞에서는 Dense 클래스의 객체 dense1, dense2를 만들어 Sequential 클래스에 전달했다. 이 두 객체를 따로 저장하여 쓸 일이 없기 때문에 다음처럼 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다.

In [35]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid',input_shape=(784,),
                     name='hidden'),
    keras.layers.Dense(10, activation='softmax',name='output')]
    , name = '패션 MNIST 모델')

위와 같이 작업하면 아래처럼 추가되는 층을 한눈에 쉽게 알아보는 장점이 있다. 모델의 이름과 달리 층의 이름은 반드시 영문이어야 한다.

In [36]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


2개의 Dense 층이 이전과 동일하게 추가되었고 파라미터 개수도 같다. 바뀐 것은 모델 이름과 층 이름이다. 여러 모델과 많은 층을 사용할 때 name 매개변수를 구분하기 쉽다. 이 방법이 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어진다. 또 조건에 따라 층을 추가할 수도 없다. Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법은 모델의 add() 메서드이다.

In [37]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid',input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

여기에서도 Dense 클래스의 객체를 따로 변수에 담지 않고 바로 add() 메서드로 전달한다. 이 방법은 한눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.

In [38]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


d이제 모델을 훈련해 보자. compile() 메서드의 설정은 1절에서 했던 것과 동일하다. 여기에서도 5번 에포크 동안 훈련해보자.

In [41]:
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
48000/48000 [==============================] - 1s 30us/sample - loss: 0.3139 - acc: 0.8857
Epoch 2/5
48000/48000 [==============================] - 1s 29us/sample - loss: 0.2996 - acc: 0.8904
Epoch 3/5
48000/48000 [==============================] - 1s 29us/sample - loss: 0.2871 - acc: 0.89470s - loss: 0.2856 - a
Epoch 4/5
48000/48000 [==============================] - 1s 31us/sample - loss: 0.2775 - acc: 0.89810s - loss: 0.2
Epoch 5/5
48000/48000 [==============================] - 1s 29us/sample - loss: 0.2674 - acc: 0.90230s - loss: 0.2652 - acc: 0. - ETA: 0s - loss: 0.2656 - acc:


훈련 세트에 대한 성능을 보면 추가된 성능을 향상시켰다는 것을 잘 알 수 있다. 인공 신경망에 몇 개의 층을 추가하더라도 compile() 메서드와 fit() 메서드의 사용법은 동일하다. 이것이 케라스 API의 장점이다.

### - 렐루 함수

초창기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수였다. 하지만 이 함수에는 단점이 있다. 이 함수의 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.

특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만든다. 이를 개선하기 위해 다른 종류의 활성화 함수가 제안되었다. 바로 렐루 함수이다. 렐루 함수는 아주 간단하다. 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다. 

렐루 함수는 max(0,z)와 같이 쓸 수 있다. 이 함수는 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력한다. 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다. 은닉층의 활성화 함수에 시그모이드 함수 대신 렐루 함수를 적용하기 전에 케라스에서 제공하는 편리한 층 하나를 더 살펴보자.

패션 MNIST 데이터는 28 x 28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼쳤다. 직접 이렇게 1차원으로 펼쳐도 좋지만 케라스에서는 이를 위한 Flatten 층을 제공한다.

사실 Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. 입력에 곱해지는 가중치나 절편이 없다. 따라서 인공 신경망의 성능을 위해 기여하는 바는 없다. 하지만 Flatten 클래스를 층처럼 입력층과 은닉층 사이에 추가하기 때문에 이를 층이라 부른다.

In [42]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [43]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


첫 번째 등장하는 Flatten 클래스에 포함된 모델 파라미터는 0개이다. 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 또 하나의 장점이다. 앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달된다는 것을 알 수 있다. 이는 이전에 만들었던 모델에서는 쉽게 눈치채기 어려웠다. 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시키는 것이 케라스 API의 철학 중 하나이다.

In [44]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [45]:
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
48000/48000 [==============================] - 2s 32us/sample - loss: 0.5292 - acc: 0.8162
Epoch 2/5
48000/48000 [==============================] - 1s 30us/sample - loss: 0.4042 - acc: 0.85411s - los
Epoch 3/5
48000/48000 [==============================] - 1s 30us/sample - loss: 0.3594 - acc: 0.86811s - los
Epoch 4/5
48000/48000 [==============================] - 1s 31us/sample - loss: 0.3313 - acc: 0.8784
Epoch 5/5
48000/48000 [==============================] - 2s 33us/sample - loss: 0.3127 - acc: 0.8855


나머지는 이전의 코드